In [1]:
# Packages
import json
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from itertools import islice
from itertools import combinations
import math
import random
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, fcluster
print("Packages Imported")

Packages Imported


In [2]:
# ======================================================================
# 1: [FUNCTIONS] Extract model words and create binary product vectors
# ======================================================================

def load_data():
    with open("TVs-all-merged.json") as file:
        data = json.load(file)

    modelIDs = []
    shops = []
    titles = []
    features = []

    for key, value_list in data.items():
        for record in value_list:
            modelIDs.append(record.get("modelID", None))
            shops.append(record.get("shop", None))
            titles.append(record.get("title", None))
            features.append(record.get("featuresMap", None))
        
    # Combine into a DataFrame
    df = pd.DataFrame({
        "modelId": modelIDs,
        "shop": shops,
        "title": titles,
        "features": features
    })

    df['uniqueId'] = df.index
    
    return df

def extract_brand(title):
    # List of known TV brands
    known_brands = ["apple", "affinity", "avue", "azend", "coby", "contex", "compaq", 
    "craig", "curtisyoung", "elite", "elo", "epson", "gpx", "haier", 
    "hannspree", "hello kitty", "hp", "dynex", "hisense", "hiteker", 
    "insignia", "jvc", "lg", "magnavox", "mitsubishi", "naxa", "nec", 
    "optoma", "panasonic", "philips", "proscan", "sceptre", "pyle", 
    "rca", "sanyo", "sansui", "seiki", "sony", "samsung", "sigmac", 
    "sharp", "sunbritetv", "supersonic", "tcl", "toshiba", "upstar", 
    "venturer", "vizio", "viewsonic", "viore", "westinghouse"
    ]
    # Check if product is from known brand
    title_lower = title.lower()
    for brand in known_brands:
        if brand in title_lower:
            return brand
    return None

def clean_title(title):
    
    title = title.replace("'", "' ").replace('"', '" ')
    
    inch_variants = ['inch', 'inches', '"', '-inch', ' inch', 'inch ', 'inch', '-inch', '”', "'"]
    for variant in inch_variants:
        title = title.replace(variant, "inch ")

    hz_variants = ['hertz', 'hz', ' hz', '-hz', 'hz']
    for variant in hz_variants:
        title = title.replace(variant, "hz ")
    
    title = title.replace("/", "").replace("-", "")
    
    title = title.lower()
    
    return title


def extract_model_words(title):
    pattern = r'\b[a-zA-Z0-9]+\b' 
    matches = re.findall(pattern, title)
    
    model_words = [word for word in matches if any(char.isdigit() for char in word) and any(char.isalpha() for char in word)]
    
    unique_model_words = list(dict.fromkeys(model_words))
    
    return unique_model_words

def create_binary_vectors(df: pd.DataFrame) -> pd.DataFrame:
    # Extract model words for each product
    df['model words'] = df['title'].apply(lambda t: extract_model_words(clean_title(t)))

    # Extract all model words from all product titles
    all_model_words = set()
    for model_words in df['model words']:
        all_model_words.update(model_words)

    model_word_list = sorted(all_model_words)

    # Use product_id as the product identifier
    uniqueIds = df['uniqueId'].tolist()

    # Construct the binary matrix
    binary_matrix = []
    for mw in model_word_list:
        binary_row = [1 if mw in model_words else 0 for model_words in df['model words']]
        binary_matrix.append(binary_row)

    binary_df = pd.DataFrame(binary_matrix, columns=uniqueIds, index=model_word_list)
    return binary_df


In [3]:
# =============================================================
# 2: [Functions] Create Signature Matrix with Min-Hashing
# =============================================================

def min_hashing(binary_matrix, n_hashes):
    """
    Performs min-hashing on the binary matrix with progress bars.
    
    Parameters:
        binary_matrix (pd.DataFrame): Binary product vectors (model words as rows, products as columns).
        p (int): Number of min-hash functions (permutations).

    Returns:
        pd.DataFrame: Signature matrix with p rows (hash functions) and columns as products.
    """
    # Set seed for reproducibility
    np.random.seed(1)
    
    n_rows, n_columns = binary_matrix.shape
    signature_matrix = np.full((n_hashes, n_columns), np.inf)
    
    # Generate random permutations of row indices
    row_indices = np.arange(n_rows)
    
    for i in range(n_hashes): 
        np.random.shuffle(row_indices)  
        for col_idx in range(n_columns):
            for perm_row_idx in row_indices:
                if binary_matrix.iloc[perm_row_idx, col_idx] == 1:  
                    signature_matrix[i, col_idx] = perm_row_idx  
                    break
    
    # Convert signature matrix to DataFrame 
    signature_df = pd.DataFrame(signature_matrix, columns=binary_matrix.columns)
    return signature_df

def verify_signature_matrix(binary_matrix, signature_matrix):
    """
    Verifies that the signature matrix approximates Jaccard Similarity.

    Parameters:
        binary_matrix (pd.DataFrame): Original binary matrix.
        signature_matrix (pd.DataFrame): Generated signature matrix.

    Returns:
        dict: Summary statistics including MAE, MSE, and correlation.
    """
    exact_similarities = []
    approx_similarities = []
    
    # Compute exact and approximate Jaccard similarities
    for i in range(signature_matrix.shape[1]):
        for j in range(i + 1, signature_matrix.shape[1]):
            vector1 = binary_matrix.iloc[:, i].values
            vector2 = binary_matrix.iloc[:, j].values
            
            # Exact Jaccard similarity
            jaccard_exact = np.sum(np.logical_and(vector1, vector2)) / np.sum(np.logical_or(vector1, vector2))
            exact_similarities.append(jaccard_exact)
            
            # Approximate Jaccard similarity
            signature1 = signature_matrix.iloc[:, i].values
            signature2 = signature_matrix.iloc[:, j].values
            jaccard_approx = np.sum(signature1 == signature2) / signature_matrix.shape[0]
            approx_similarities.append(jaccard_approx)
    
    # Convert to numpy arrays for calculations
    exact_similarities = np.array(exact_similarities)
    approx_similarities = np.array(approx_similarities)
    
    # Calculate summary metrics
    mae = np.mean(np.abs(exact_similarities - approx_similarities))
    mse = np.mean((exact_similarities - approx_similarities) ** 2)
    correlation = np.corrcoef(exact_similarities, approx_similarities)[0, 1]
    
    return {
        "Mean Absolute Error (MAE)": mae,
        "Mean Squared Error (MSE)": mse,
        "Correlation": correlation,
        "Exact Similarities Mean": np.mean(exact_similarities),
        "Approx Similarities Mean": np.mean(approx_similarities)
    }

In [4]:
# =============================================================
# 3: [Functions] Apply LSH to find Nearest Neighbours
# =============================================================
def pad_signature_matrix(signature_matrix, target_rows=None):
    """
    Pads the signature matrix to ensure the number of rows is divisible by common values of r.

    Parameters:
        signature_matrix (pd.DataFrame): Original signature matrix.
        target_rows (int, optional): Desired total number of rows. If None, calculate the next multiple of 10.

    Returns:
        pd.DataFrame: Padded signature matrix.
    """
    current_rows, current_columns = signature_matrix.shape

    if target_rows is None:
        # Find the next multiple of 10 greater than the current number of rows
        target_rows = (current_rows + 9) // 10 * 10  # Round up to the nearest multiple of 10

    if target_rows < current_rows:
        raise ValueError("Target rows cannot be smaller than the current number of rows.")

    # Calculate the number of rows to add
    padding_rows = target_rows - current_rows
    if padding_rows > 0:
        # Create a DataFrame of padding rows with np.inf
        padding = pd.DataFrame(np.inf, index=range(padding_rows), columns=signature_matrix.columns)
        signature_matrix = pd.concat([signature_matrix, padding], ignore_index=True)

    print(f"Padded signature matrix to {target_rows} rows.")
    return signature_matrix

def find_b_r_from_t(n, t, max_padding_allowed=15):
    """
    Given n, t, and a maximum allowed padding in terms of rows,
    find (b, r) that yields (1/b)^(1/r) closest to t, with b*r - n <= max_padding_allowed.
    """
    best_diff = float('inf')
    best_b = None
    best_r = None
    best_padding = None

    for r in range(1, n+1):
        b = math.ceil(n / r)
        b_r = b * r
        padding = b_r - n
        if padding <= max_padding_allowed:
            t_prime = (1/b)**(1/r)
            diff = abs(t_prime - t)
            if diff < best_diff or (diff == best_diff and (best_padding is None or padding < best_padding)):
                best_diff = diff
                best_b = b
                best_r = r
                best_padding = padding

    # If still None, no pair met the constraint, so choose best ignoring it
    if best_b is None:
        best_diff = float('inf')
        for r in range(1, n+1):
            b = math.ceil(n / r)
            t_prime = (1/b)**(1/r)
            diff = abs(t_prime - t)
            if diff < best_diff:
                best_diff = diff
                best_b = b
                best_r = r

    return best_b, best_r, best_diff
    

def apply_lsh(signature_matrix, t):
    """
    Apply LSH given a desired threshold t.
    This function:
    1. Determines n from the signature_matrix.
    2. Finds suitable (b, r) from t.
    3. Applies LSH with those parameters.
    
    Returns:
    - candidate_pairs: set of (product1, product2)
    - b, r: chosen parameters
    - t_actual: the t' achieved by the chosen pair (1/b)^(1/r)
    """
    # Convert signature_matrix to numpy if needed
    if isinstance(signature_matrix, np.ndarray):
        pass
    else:
        signature_matrix = signature_matrix.values

    n, N = signature_matrix.shape
    b, r, _ = find_b_r_from_t(n, t)
    # Now we have b and r. Compute t' for reference
    t_actual = (1/b)**(1/r)

    # Pad the signature matrix if necessary
    new_n = b * r
    if new_n > n:
        pad_rows = new_n - n
        # Use a stable padding value, e.g., -1
        pad_matrix = np.full((pad_rows, N), -1)
        signature_matrix = np.vstack([signature_matrix, pad_matrix])
        n = new_n

    candidate_pairs = set()
    start = 0
    for band_idx in range(b):
        end = start + r
        band = signature_matrix[start:end, :]

        buckets = {}
        for col_idx in range(N):
            band_slice = tuple(band[:, col_idx])
            if band_slice not in buckets:
                buckets[band_slice] = []
            buckets[band_slice].append(col_idx)

        # Extract candidate pairs from buckets
        for bucket_items in buckets.values():
            if len(bucket_items) > 1:
                for i in range(len(bucket_items)):
                    for j in range(i+1, len(bucket_items)):
                        p1 = bucket_items[i]
                        p2 = bucket_items[j]
                        candidate_pairs.add((min(p1,p2), max(p1,p2)))

        start = end

    return candidate_pairs, b, r, t_actual



In [14]:
# ====================================================================================================================
# 4: [Functions] Apply MSM to find duplicate products
# **---------------------------------------------------
# ** [Ref Paper] Van Bezu, et al. (2015, April). Multi-component similarity method for web product duplicate detection.
# =====================================================================================================================
def calculate_qgram_similarity(str1, str2, q=3):
    def qgrams(s, length):
        return {s[i:i+length] for i in range(len(s)-length+1)} if len(s) >= length else {s}
    
    qg1 = qgrams(str1.lower(), q)
    qg2 = qgrams(str2.lower(), q)
    
    intersect = qg1.intersection(qg2)
    union = qg1.union(qg2)
    return len(intersect) / len(union) if len(union) > 0 else 0.0

def is_same_shop(p1,p2, df):
    return df.at[p1, 'shop'] == df.at[p2, 'shop']
    
def is_different_brand(p1,p2,df):
    return df.at[p1, 'brand'] != df.at[p2, 'brand']

def extract_model_words_features(value):
    pattern = re.compile(r'[a-zA-Z0-9]*(([0-9]+[^0-9, ]+)|([^0-9, ]+[0-9]+))[a-zA-Z0-9]*')
    matches = re.findall(pattern, value.lower())
    
    mw = [word for word in matches if any(char.isdigit() for char in word) and any(char.isalpha() for char in word)]
    unique_mw = list(dict.fromkeys(mw))
    
    return unique_mw

def extract_mw_features(features):
    mw = set()
    for val in features.values():
        mw.update(extract_model_words_features(val))
    return mw
    
    
def calculate_mwPerc(C,D):
    if len(C) == 0 or len(D)==0:
        return 0.0
    intersect = C.intersection(D)
    union = C.union(D)
    return len(inter)/len(union) if len(uni)>0 else 0.0
    
def calculate_titleSim(tmw1, tmw2):
    if len(tmw1) == 0 or len(tmw2) == 0:
        return 0.0
    intersection = tmw1.intersection(tmw2)
    union = tmw1.union(tmw2)
    return len(intersection) / len(union) if union else 0.0

def build_dissimilarity_matrix_MSM(df, candidate_pairs, alpha=0.602, gamma=0.756, mu=0.650):
    # Initialise dissimilarity matrix 
    N = len(df)
    inf_distance = 500
    diss_matrix = np.ones((N,N)) * inf_distance
    np.fill_diagonal(diss_matrix, 0)

    for (pi,pj) in candidate_pairs:
        # Checks beforehand
        if pi == pj:
            continue
        if is_same_shop(pi,pj,df) or is_different_brand(pi,pj,df):
            diss_matrix[pi, pj] = inf_distance
            diss_matrix[pj, pi] = inf_distance
            continue

        # Extract KVPs
        KVPi = df.at[pi, 'features']
        KVPj = df.at[pj, 'features']

        # Initialise starting variables
        sim = 0.0
        avgSim = 0.0
        m = 0
        w = 0.0

        nmki = dict(KVPi)
        nmkj = dict(KVPj)

        #(1) Calculate avgSim of KVP
        for s_key, s_value in KVPi.items():
            for t_key, t_value in KVPj.items():
                keySim = calculate_qgram_similarity(s_key, t_key, q=3)
                if keySim > gamma:
                    valueSim = calculate_qgram_similarity(s_value, t_value, q=3)
                    weight = keySim
                    sim += weight * valueSim
                    m += 1
                    w += weight
                    nmki.pop(s_key, None)
                    nmkj.pop(t_key, None)
        avgSim = (sim/m) if w>0 else 0.0
        
        #(2) Calculate mwPerc from non-matching keys
        mw_i = extract_mw_features(nmki)
        mw_j = extract_mw_features(nmkj)
        mwPerc = calculate_mwPerc(mw_i,mw_j)

        #(3) Calculate titleSim with TMWM
        tmwi = set(df.at[pi, 'model words'])
        tmwj = set(df.at[pj, 'model words'])
        titleSim = calculate_titleSim(tmwi, tmwj)
        
        # Combine to obtain MSM 
        minFeatures = min(len(KVPi), len(KVPj))
        if titleSim == -1:
            theta1 = m/minFeatures
            theta2 = 1 - theta1
            hSim = theta1 * avgSim + theta2 * mwPerc
        else:
            theta1 = (1-mu) * (m/minFeatures)
            theta2 = 1 - mu - theta1
            hSim = theta1 * avgSim + theta2 * mwPerc + mu * titleSim 

        diss = 1 - hSim
        diss_matrix[pi,pj] = diss
        diss_matrix[pj,pi] = diss

    return diss_matrix 

def apply_hClustering(diss_matrix, linkage='single', epsilon=0.52): 
    model = AgglomerativeClustering(distance_threshold=epsilon,
                                    n_clusters=None,
                                    linkage=linkage,
                                    metric='precomputed')
    model.fit(diss_matrix)

    N = len(df)
    inf_distance = 500
    n_comparisons = np.zeros((N, N))
    for i, j in np.argwhere((diss_matrix > 0) & (diss_matrix < inf_distance)):
        n_comparisons[min(i, j), max(i, j)] = 1
    n_comparisons = n_comparisons.sum()
    
    return model


In [7]:
# =============================================================
# 5: [Functions] Evaluation Metrics
# =============================================================

def true_duplicates_found(unique_clusters, df):
    found_duplicates = 0
    
    for c in tqdm(unique_clusters, desc="Check true clusters"):
        members = np.where(clusters==c)[0]
        #print(f"members to check: {members}")
        for i, j in combinations(members, 2):
            if df.at[i, 'modelId'] == df.at[j, 'modelId']:
                found_duplicates += 1
                
    return found_duplicates

def true_duplicates_found_CandidateDuplicates(candidates, df):
    found_duplicates = 0
    
    for i,j in candidates:
        if df.at[i, 'modelId'] == df.at[j, 'modelId']:
            found_duplicates += 1
                
    return found_duplicates

def compute_total_duplicates(df):
    """
    Compute the total number of true duplicate pairs (Dn) in the dataset.
    Two products are true duplicates if they share the same modelId.
    """
    groups = df.groupby("modelId")
    Dn = 0
    for _, group_indices in groups.groups.items():
        size = len(group_indices)
        if size > 1:
            Dn += size * (size - 1) // 2
    return Dn

def count_true_duplicates_in_pairs(pairs, df):
    """
    Count how many of the given pairs are true duplicates.
    pairs: iterable of (i,j)
    df: DataFrame with 'modelId' column
    """
    found_duplicates = 0
    for (i, j) in pairs:
        if df.at[i, 'modelId'] == df.at[j, 'modelId']:
            found_duplicates += 1
    return found_duplicates

def count_true_duplicates_in_clusters(clusters, df):
    """
    Count how many duplicate pairs are found within the identified clusters.
    clusters: array of cluster labels for each product
    df: DataFrame with 'modelId'
    """
    found_duplicates = 0
    unique_clusters = np.unique(clusters)
    for c in unique_clusters:
        members = np.where(clusters == c)[0]
        # Count how many duplicate pairs among these members
        for i, j in combinations(members, 2):
            if df.at[i, 'modelId'] == df.at[j, 'modelId']:
                found_duplicates += 1
    return found_duplicates

def compute_metrics_LSH(Df_LSH, Nc, Dn):
    """
    Compute PQ, PC, and F1 for LSH:
    PQ = Df_LSH / Nc
    PC = Df_LSH / Dn
    F1 = 2 * (PQ*PC)/(PQ+PC)
    """
    if Nc == 0:
        return 0.0, 0.0, 0.0  # no comparisons made
    PQ = Df_LSH / Nc
    if Dn == 0:
        # If no duplicates in dataset, PC and F1 not meaningful
        return PQ, 0.0, 0.0
    PC = Df_LSH / Dn
    if (PQ + PC) == 0:
        F1 = 0.0
    else:
        F1 = 2 * PQ * PC / (PQ + PC)
    return PQ, PC, F1

def compute_metrics_MSM(Df_MSM, N_MSM, Dn):
    """
    Compute precision, recall, and F1 for MSM:
    precision = Df_MSM / N_MSM (duplicates flagged by MSM)
    recall = Df_MSM / Dn
    F1 = 2 * (precision*recall)/(precision+recall)
    N_MSM = number of pairs flagged as duplicates by MSM
            i.e. pairs from the clusters formed by MSM.

    If you consider all pairs in the same cluster as "flagged duplicates," then N_MSM is:
      sum over all clusters: number_of_pairs_in_that_cluster
    """
    if N_MSM == 0:
        return 0.0, 0.0, 0.0
    precision = Df_MSM / N_MSM
    if Dn == 0:
        # If no duplicates in dataset, recall and F1 not meaningful
        return precision, 0.0, 0.0
    recall = Df_MSM / Dn
    if (precision + recall) == 0:
        F1 = 0.0
    else:
        F1 = 2 * precision * recall / (precision + recall)
    return precision, recall, F1

def count_flagged_duplicates_by_MSM(clusters):
    """
    Count how many pairs are flagged as duplicates by MSM clustering.
    This means counting all pairs from each cluster, since MSM considers 
    all products in a cluster as duplicates.
    """
    flagged_duplicates = 0
    unique_clusters = np.unique(clusters)
    for c in unique_clusters:
        members = np.where(clusters == c)[0]
        size = len(members)
        if size > 1:
            flagged_duplicates += size * (size - 1) // 2
    return flagged_duplicates

In [2]:
# ====== RUN =================================================

bootstrap = 10
n_train = 1000
t_range = [0.05, 0.10, 0.20, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
epsilon_range = [0.40, 0.45, 0.50, 0.55, 0.60, 0.75, 0.80]
linkage_methods = ['single', 'average', 'complete']

df = load_data()
df["brand"] = df["title"].apply(extract_brand)
df["title"] = df["title"].apply(clean_title)
df["model words"] = df["title"].apply(extract_model_words)

N = df.shape[0]

all_results=[]

random.seed(1)

for bootstrap in tqdm(range(n_bootstraps), desc="Bootstraps"):
    # Split in training and test set
    train_indices = random.sample(range(N), n_train) 
    train_set = sorted(set(train_indices))
    test_indices = [i for i in range(N) if i not in train_set]
    
    df_train = df.iloc[train_set].reset_index(drop=True).copy()
    df_test = df.iloc[test_indices].reset_index(drop=True).copy()
    
    # Recalculate uniqueId for consistency after resetting the index
    df_train['uniqueId'] = df_train.index
    df_test['uniqueId'] = df_test.index

    # Train metrics
    N_train = len(df_train)
    Dn_train = compute_total_duplicates(df_train)
    total_pairs_train = N_train * (N_train - 1) / 2
    
    # Test metrics
    N_test = len(df_test)
    Dn_test = compute_total_duplicates(df_test)
    total_pairs_test = N_test * (N_test - 1) / 2

    # ==== TRAINING ====
    # Bulild training set
    binary_train = create_binary_vectors(df_train)
    n_hashes_train = math.floor(df_train.shape[0] * 0.50)
    signature_train = min_hashing(binary_train, n_hashes_train)

    # Initialisiation
    best_F1_train = {'single': -1, 'average': -1, 'complete': -1}
    best_eps_train = {'single': None, 'average': None, 'complete': None}

    for t in tqdm(t_range, desc="[Training] Iterating through t-values"):

        # Apply LSH and evaluate
        candidates, b, r, t_actual = apply_lsh(signature_train, t)
        Nc = len(candidates)
        Df_LSH = count_true_duplicates_in_pairs(candidates, df_train)
        PQ, PC, F1_LSH = compute_metrics_LSH(Df_LSH, Nc, Dn_train)
        fraction_of_comparisons = Nc / total_pairs_train

        # Build dissimilarity for MSM
        dist_matrix = build_dissimilarity_matrix_MSM(df_train, candidates, alpha=0.602, gamma=0.756, mu=0.650)
        
        for eps in epsilon_range:

            for linkage in linkage_methods: 
                # Apply MSM clustering and evaluate
                model = apply_hClustering(dist_matrix, linkage=linkage, epsilon=eps)
                cluster_labels = model.labels_
                
                Df_MSM = count_true_duplicates_in_clusters(cluster_labels, df_train)
                N_MSM = count_flagged_duplicates_by_MSM(cluster_labels)
                precision, recall, F1_MSM = compute_metrics_MSM(Df_MSM, N_MSM, Dn_train)

                # Update best_epsilon based on F1 of MSM
                if F1_MSM > best_F1_train[linkage]:
                    best_F1_train[linkage] = F1_MSM
                    best_eps_train[linkage] = eps

                # Store results 
                all_results.append({
                    'bootstrap': bootstrap,
                    'bootstrap_sort': 'training',
                    'b': b,
                    'r': r,
                    't': t_actual,
                    'epsilon': eps,
                    'linkage_method': linkage,
                    'Nc': Nc,
                    'Dn': Dn_train,
                    'Df_LSH': Df_LSH,
                    'PQ_LSH': PQ,
                    'PC_LSH': PC,
                    'F1_LSH': F1_LSH,
                    'fraction_of_comparisons': fraction_of_comparisons,
                    'Df_MSM': Df_MSM,
                    'N_MSM': N_MSM,
                    'precision_MSM': precision,
                    'recall_MSM': recall,
                    'F1_MSM': F1_MSM
                })

    
    # ==== TESTING ====
    # Bulild testing set 
    binary_test = create_binary_vectors(df_test)
    n_hashes_test = math.floor(df_test.shape[0] * 0.5)
    signature_test = min_hashing(binary_test, n_hashes_test)
    
    for t in tqdm(t_range, desc="[Testing] Iterating through t-values"):
        candidates, b, r, t_actual = apply_lsh(signature_test, t)
        Nc = len(candidates)
        Df_LSH = count_true_duplicates_in_pairs(candidates, df_test)
        PQ, PC, F1_LSH = compute_metrics_LSH(Df_LSH, Nc, Dn_test)
        fraction_of_comparisons = Nc / total_pairs_test
    
        # Build dissimilarity matrix MSM
        dist_matrix = build_dissimilarity_matrix_MSM(df_test, candidates, alpha=0.602, gamma=0.756, mu=0.650)
            
        for linkage in linkage_methods: 
            best_epsilon = best_eps_train[linkage]
            if best_epsilon is not None:
                model = apply_hClustering(dist_matrix, linkage=linkage, epsilon=best_epsilon)
                cluster_labels = model.labels_

                Df_MSM = count_true_duplicates_in_clusters(cluster_labels, df_test)
                N_MSM = count_flagged_duplicates_by_MSM(cluster_labels)
                precision, recall, F1_MSM = compute_metrics_MSM(Df_MSM, N_MSM, Dn_test)

                # Append test results
                all_results.append({
                    'bootstrap': bootstrap,
                    'bootstrap_sort': 'test',
                    'b': b,
                    'r': r,
                    't': t_actual,
                    'epsilon': best_epsilon,
                    'linkage_method': linkage,
                    'Nc': Nc,
                    'Dn': Dn_test,
                    'Df_LSH': Df_LSH,
                    'PQ_LSH': PQ,
                    'PC_LSH': PC,
                    'F1_LSH': F1_LSH,
                    'fraction_of_comparisons': fraction_of_comparisons,
                    'Df_MSM': Df_MSM,
                    'N_MSM': N_MSM,
                    'precision_MSM': precision,
                    'recall_MSM': recall,
                    'F1_MSM': F1_MSM
                })
                
df_results = pd.DataFrame(all_results)
df_results.to_excel("evaluation_results_bootstrap.xlsx", index=False)
print("Results saved to evaluation_results_bootstrap.xlsx")

NameError: name 'load_data' is not defined